In [11]:
import yaml
import numpy as np
import zarr
import os

In [10]:
root_gt_path = "/nrs/saalfeld/heinrichl/data/cellmap_labels/fly_organelles/"

In [18]:
with open("../data.yaml") as f:
    data = yaml.safe_load(f)
with open("../label.yaml") as f:
    label = yaml.safe_load(f)

In [5]:
for lbl, sublbls in label.items():
    print(lbl, sublbls)

nuc ['ne_mem', 'ne_lum', 'np_out', 'np_in', 'hchrom', 'nhchrom', 'echrom', 'nechrom', 'nucpl', 'nucleo']
golgi ['golgi_mem', 'golgi_lum']
ves ['ves_mem', 'ves_lum']
endo ['endo_mem', 'endo_lum']
lyso ['lyso_mem', 'lyso_lum']
ld ['ld_mem', 'ld_lum']
eres ['eres_mem', 'eres_lum']
perox ['perox_mem', 'perox_lum']
mito ['mito_mem', 'mito_lum', 'mito_ribo']
ne ['ne_mem', 'ne_lum', 'np_out', 'np_in']
np ['np_out', 'np_in']
chrom ['hchrom', 'nhchrom', 'echrom', 'nechrom']
mt ['mt_out', 'mt_in']
isg ['isg_mem', 'isg_lum', 'isg_ins']
er_mem_all ['er_mem', 'eres_mem', 'ne_mem']
ne_mem_all ['ne_mem', 'np_out', 'np_in']
cent_all ['cent', 'cent_dapp', 'cent_sdapp']
chloroplast ['chloroplast_lum', 'chloroplast_mem', 'chloroplast_sg']
vacuole ['vacuole_lum', 'vacuole_mem']
er ['er_mem', 'er_lum', 'eres_mem', 'eres_lum', 'ne_mem', 'ne_lum', 'np_out', 'np_in', 'er_mem_all', 'ne_mem_all']
cell ['pm', 'mito_mem', 'mito_lum', 'mito_ribo', 'golgi_mem', 'golgi_lum', 'ves_mem', 'ves_lum', 'endo_mem', 'endo_l

In [23]:
for ds_dir, ds_info in data.items():
    crops = []
    for crop in ds_info["crops"]:
        crops.extend(crop.split(","))
    for crop in crops:
        path = os.path.join(root_gt_path, ds_dir, "groundtruth.zarr", crop)
        crop_zarr = zarr.open(path, "r")
        annotated = crop_zarr.attrs["cellmap"]["annotation"]["class_names"]
        cans = []
        cants = []
        for lbl, sublbls in label.items():
            if all(sublbl in annotated for sublbl in sublbls):
                cans.append(lbl)
                annotated.append(lbl)
            else:
                cants.append(lbl)
        print(f"{crop}, cans: {cans}, cants: {cants}")
        #for lbl, sublbs in label.items():
            


crop234, cans: ['nuc', 'golgi', 'endo', 'lyso', 'ld', 'eres', 'perox', 'ne', 'np', 'chrom', 'mt', 'er_mem_all', 'ne_mem_all', 'cent_all', 'er'], cants: ['ves', 'mito', 'isg', 'chloroplast', 'vacuole', 'cell']
crop236, cans: ['nuc', 'golgi', 'endo', 'lyso', 'ld', 'eres', 'perox', 'ne', 'np', 'chrom', 'mt', 'er_mem_all', 'ne_mem_all', 'cent_all', 'er'], cants: ['ves', 'mito', 'isg', 'chloroplast', 'vacuole', 'cell']
crop237, cans: ['nuc', 'golgi', 'endo', 'lyso', 'ld', 'eres', 'perox', 'ne', 'np', 'chrom', 'mt', 'er_mem_all', 'ne_mem_all', 'cent_all', 'er'], cants: ['ves', 'mito', 'isg', 'chloroplast', 'vacuole', 'cell']
crop239, cans: ['nuc', 'golgi', 'endo', 'lyso', 'ld', 'eres', 'perox', 'ne', 'np', 'chrom', 'mt', 'er_mem_all', 'ne_mem_all', 'cent_all', 'er'], cants: ['ves', 'mito', 'isg', 'chloroplast', 'vacuole', 'cell']
crop248, cans: ['nuc', 'golgi', 'endo', 'lyso', 'ld', 'eres', 'perox', 'ne', 'np', 'chrom', 'mt', 'er_mem_all', 'ne_mem_all', 'cent_all', 'er'], cants: ['ves', 'mit